###Clustering Businesses

In [2]:
import os
import csv
import re

business_csv_file = open(os.path.join("..","data","biz_csv","business.csv"),"r", encoding='utf-8')
business_csv = csv.reader(business_csv_file)

categories = list()
for line in business_csv:
    #print(line)
    categories.append(line[-1])
categories = categories[1:]

business_csv_file.close()

unique_cats = set()
for category in categories:
    cat_list = re.split(",|&",category)
    for cat in cat_list:
        cat = cat.lower().strip()
        if len(cat) > 0:
            unique_cats.add(cat.lower().strip())

print("There are "+str(len(unique_cats))+" unique categories and subcategories for all businesses")

There are 1310 unique categories and subcategories for all businesses


In [12]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import csv
import pandas


# business_csv_file = open(os.path.join("..","data","biz_csv","business.csv"),"r", encoding="utf8")
# business_csv = csv.reader(business_csv_file)

def cluster_by_category(data, n_clusters):
    categories = list()
    for line in data:
        categories.append(line[-1])
        
    vectorizer = TfidfVectorizer(stop_words='english') #,token_pattern='[a-zA-Z0-9\s&]+' # term-frequency x inverse-document frequency
    # tokenize based on comma instead of space. Otherwise, words like "Public Services" will not be seen as a single word 
    # by the vectorizer
    cat = vectorizer.fit_transform(categories)

    kmeans_model = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=100, n_init=1,random_state=1)
    kmeans_model.fit(cat)

    business_csv_clustered_file = open(os.path.join("..","data","biz_csv","business_clustered.csv"),"a+", newline='')
    business_clustered_csv = csv.writer(business_csv_clustered_file)

    for line in data:
        if not (line[-1] == "categories"):
            line.extend(kmeans_model.predict(vectorizer.transform(line[-1:])))
            business_clustered_csv.writerow(line)

    business_csv_clustered_file.close()

In [10]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import csv
import pandas

def cluster_by_position_and_category(n_clusters):
    business_csv_file = open(os.path.join("..","data","biz_csv","business.csv"),"r", encoding="utf8")
    business_csv = csv.reader(business_csv_file)

    categories = list()
    first_line = 1;
    for line in business_csv:
        if first_line > 1:
            categories.append([float(line[1]), float(line[2])])
        else:
            categories.append([line[1], line[2]])
        first_line += 1
    categories = categories[1:]

    kmeans_model = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=100, n_init=1,random_state=1)
    kmeans_model.fit(categories)

    business_csv_file.close()

    business_csv_file = open(os.path.join("..","data","biz_csv","business.csv"),"r", encoding="utf8")
    business_csv = csv.reader(business_csv_file)

    cluster_buckets = dict()

    for line in business_csv:
        if not (line[-1] == "categories"):
            pred = kmeans_model.predict([[line[1],line[2]]])[0]
            if not pred in cluster_buckets: 
                cluster_buckets[pred] = list()
            cluster_buckets[pred].append(line)

    business_csv_file.close()
    for cluster in cluster_buckets:
        print("Processing cluster number "+str(cluster))
        cluster_by_category(cluster_buckets[cluster], n_clusters=min(len(cluster_buckets[cluster]),n_clusters))

    print("Done")

In [11]:
cluster_by_position_and_category(20)

Processing cluster number 16
Processing cluster number 6
Processing cluster number 5
Processing cluster number 4
Processing cluster number 10
Processing cluster number 19
Processing cluster number 17
Processing cluster number 1
Processing cluster number 2
Processing cluster number 8
Processing cluster number 7
Processing cluster number 11
Processing cluster number 3
Processing cluster number 0
Processing cluster number 15
Processing cluster number 9
Processing cluster number 18
Processing cluster number 12
Processing cluster number 13
Processing cluster number 14
Done


In [13]:
#cross-validation 50-25-25 over this business.csv (cluster only 50% data)
#do the regression, training the check-in data over 50% data. --> in RAM! 
#for each validation data, we will find the cluster and then filter businesses fromm the same cluster
#take check-in data for those check-in businesses and learn to predict the crowd